In [2]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE  

                                                       1. DATA COLLECTION AND PREPARATIONS

In [3]:
#Load data 
data = pd.read_csv("/media/ashley/5011e0b1-5fd0-4a93-b3f6-892e8cb26354/IBM Transactions for Anti Money Laundering (AML)/HI-Large_Trans.csv", nrows=5_000)
data

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,2022/08/01 00:17,20,800104D70,20,800104D70,6794.63,US Dollar,6794.63,US Dollar,Reinvestment,0
1,2022/08/01 00:02,3196,800107150,3196,800107150,7739.29,US Dollar,7739.29,US Dollar,Reinvestment,0
2,2022/08/01 00:17,1208,80010E430,1208,80010E430,1880.23,US Dollar,1880.23,US Dollar,Reinvestment,0
3,2022/08/01 00:03,1208,80010E650,20,80010E6F0,73966883.00,US Dollar,73966883.00,US Dollar,Cheque,0
4,2022/08/01 00:02,1208,80010E650,20,80010EA30,45868454.00,US Dollar,45868454.00,US Dollar,Cheque,0
...,...,...,...,...,...,...,...,...,...,...,...
4995,2022/08/01 00:20,2954,800AE6A00,240178,83B7DDD20,183.01,US Dollar,183.01,US Dollar,Credit Card,0
4996,2022/08/01 00:12,11848,800AE6BE0,2205341,851944130,111.44,US Dollar,111.44,US Dollar,Credit Card,0
4997,2022/08/01 00:10,952,800AE7480,952,800AE7480,3.49,US Dollar,3.49,US Dollar,Reinvestment,0
4998,2022/08/01 00:21,11304,800AE78C0,11304,800AE78C0,23.18,US Dollar,23.18,US Dollar,Reinvestment,0


In [12]:
data.head()

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,2022/08/01 00:17,20,800104D70,20,800104D70,6794.63,US Dollar,6794.63,US Dollar,Reinvestment,0
1,2022/08/01 00:02,3196,800107150,3196,800107150,7739.29,US Dollar,7739.29,US Dollar,Reinvestment,0
2,2022/08/01 00:17,1208,80010E430,1208,80010E430,1880.23,US Dollar,1880.23,US Dollar,Reinvestment,0
3,2022/08/01 00:03,1208,80010E650,20,80010E6F0,73966883.00,US Dollar,73966883.00,US Dollar,Cheque,0
4,2022/08/01 00:02,1208,80010E650,20,80010EA30,45868454.00,US Dollar,45868454.00,US Dollar,Cheque,0


In [13]:
# Check for null values using different functions
data.isna().sum()
data.info() # Use info() to get summary information


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Timestamp           5000 non-null   object 
 1   From Bank           5000 non-null   int64  
 2   Account             5000 non-null   object 
 3   To Bank             5000 non-null   int64  
 4   Account.1           5000 non-null   object 
 5   Amount Received     5000 non-null   float64
 6   Receiving Currency  5000 non-null   object 
 7   Amount Paid         5000 non-null   float64
 8   Payment Currency    5000 non-null   object 
 9   Payment Format      5000 non-null   object 
 10  Is Laundering       5000 non-null   int64  
dtypes: float64(2), int64(3), object(6)
memory usage: 429.8+ KB


In [14]:
data.describe()

,From Bank,To Bank,Amount Received,Amount Paid,Is Laundering
count,5000.000000,5.000000e+03,5.000000e+03,5.000000e+03,5000.0
mean,4037.210800,2.214049e+04,1.397458e+06,1.397458e+06,0.0
std,17728.269488,1.576202e+05,2.308181e+07,2.308181e+07,0.0
min,0.000000,0.000000e+00,1.000000e-02,1.000000e-02,0.0
25%,20.000000,2.000000e+01,2.041000e+01,2.041000e+01,0.0
50%,1046.000000,1.208000e+03,2.792185e+03,2.792185e+03,0.0
75%,2597.000000,2.776000e+03,3.412672e+04,3.412672e+04,0.0
max,312834.000000,2.210087e+06,9.913775e+08,9.913775e+08,0.0


In [15]:
#Split data set into features (X) and labels (Y)
X = data.drop('To Bank',axis = 1) #Features
Y = data['To Bank'] #labels(0 for legitimate, 1 for fraudulen)

                                                        2. FEATURE ENGINEERING

In [16]:
# Calculate the number of transactions in the past X minutes
window_size = 60  # Example: 1 hour window
transaction_data['transactions_within_window'] =data['timestamp'].apply(
    lambda ts: ((data['timestamp'] > ts - window_size) & (data['timestamp'] < ts + window_size)).sum()
)

KeyError: 'timestamp'

In [ ]:
# Compare the transaction's location with the customer's usual location
transaction_data['unusual_location'] = (transaction_data['location'] != customer_data['usual_location']).astype(int)

NameError: name 'transaction_data' is not defined

In [ ]:
# Extract time-based features
transaction_data['hour_of_day'] = pd.to_datetime(transaction_data['timestamp'], unit='s').dt.hour
transaction_data['day_of_week'] = pd.to_datetime(transaction_data['timestamp'], unit='s').dt.dayofweek

In [ ]:
# Calculate customer-specific features
customer_data = transaction_data.groupby('customer_id').agg({
    'amount': ['mean', 'std'],
    'transaction_type': lambda x: x.nunique(),
    'timestamp': ['count', 'mean']
})
customer_data.columns = ['avg_amount', 'std_amount', 'unique_transaction_types', 'transaction_count', 'avg_transaction_time']

In [ ]:
from scipy.stats import benford

# Calculate the distribution of leading digits
transaction_data['leading_digit'] = transaction_data['amount'].apply(lambda x: int(str(x)[0]))
expected_distribution = benford.pmf(range(1, 10))
observed_distribution = transaction_data['leading_digit'].value_counts(normalize=True).sort_index()

# Calculate the Earth Mover's Distance between observed and expected distributions
emd = np.sum(np.abs(expected_distribution - observed_distribution))

In [ ]:
# Aggregate features over different time windows
time_windows = [1, 7, 30]  # Example: Daily, weekly, and monthly windows
for window in time_windows:
    transaction_data[f'avg_amount_{window}d'] = transaction_data['amount'].rolling(window, min_periods=1).mean()

                                                                3. DATA SPLITTING

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

                                                                4. MODEL SELECTION AND TRAINING

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Initialize and train the model
model = RandomForestClassifier(random_state=0)
model.fit(X_train, y_train)

                                                                5. MODEL EVALUATION

In [ ]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

                                                                6. THRESHOLD AND ALERTS

In [ ]:
# For example, define a threshold for high-risk transactions
threshold = 0.7

# Predict probabilities
y_proba = model.predict_proba(X_new)[:, 1]

# Generate alerts for transactions above the threshold
for i, prob in enumerate(y_proba):
    if prob > threshold:
        generate_alert(transaction_data[i])